To perform transfer learning with ResNet-18 using Hugging Face tools on the Cats and Dogs dataset, you can use the `transformers` and `datasets` libraries for model setup and data preprocessing. Here’s a guide to set this up:

### 1. Install Necessary Packages
Ensure you have the Hugging Face `transformers` and `datasets` libraries installed:

### 2. Load the Dataset
You can use the `datasets` library to load and preprocess the dataset from Kaggle.

In [17]:
from datasets import load_dataset
from torchvision import transforms

# Load only the dataset without predefined train/test splits
dataset = load_dataset(
    "imagefolder",
    data_dir="../../data/cats_and_dogs/PetImages",
    split="train"  # Load the entire dataset initially
).train_test_split(test_size=0.2)  # Split into 80% train and 20% test

# Define the transformations
transform = transforms.Compose([
    transforms.Resize(130),
    transforms.CenterCrop(128),
    transforms.ToTensor()
])

# Apply transformations to each image
def preprocess_images(example):
    example['image'] = transform(example['image'])
    return example

# Map transformations onto both train and test splits
dataset['train'] = dataset['train'].map(preprocess_images, batched=False)
dataset['test'] = dataset['test'].map(preprocess_images, batched=False)


Resolving data files:   0%|          | 0/25000 [00:00<?, ?it/s]

Map:   0%|          | 0/19998 [00:00<?, ? examples/s]

/Users/jbaggett/miniforge3/envs/DS776v2/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]



### 3. Load the ResNet-18 Model from Hugging Face
Using `transformers`, load the ResNet-18 model and modify its classification head.


In [19]:
from transformers import AutoModelForImageClassification
import torch

# Load a ResNet-18 model pretrained on ImageNet
model = AutoModelForImageClassification.from_pretrained("microsoft/resnet-18")

# Change the classification head to output 2 classes
model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, 2)



### 4. Training Loop
You can set up a training loop or use the Hugging Face `Trainer` API to simplify training. Here’s an example using `Trainer`:

In [21]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm

# Set up data loaders
train_loader = DataLoader(dataset['train'], batch_size=16, shuffle=True)
eval_loader = DataLoader(dataset['test'], batch_size=16)

# Set model to device
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=1e-4)

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        # Get data and move to device
        inputs = batch['image'].to(device)
        labels = batch['label'].to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = torch.nn.functional.cross_entropy(outputs.logits, labels)
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        train_loss += loss.item()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1} Train Loss: {avg_train_loss:.4f}")
    
    # Evaluation loop
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in eval_loader:
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device)
            
            outputs = model(inputs)
            loss = torch.nn.functional.cross_entropy(outputs.logits, labels)
            eval_loss += loss.item()
            
            # Get predictions
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
    
    avg_eval_loss = eval_loss / len(eval_loader)
    accuracy = correct / len(dataset['test'])
    print(f"Epoch {epoch+1} Eval Loss: {avg_eval_loss:.4f}, Accuracy: {accuracy:.4f}")


Training Epoch 1:   0%|          | 0/1250 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'to'



This setup allows you to perform transfer learning on ResNet-18 using Hugging Face tools, with the desired resizing, center cropping, and modified classification head for binary classification.